In [9]:
import numpy as np

# patch for backward compatibility
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_
    
import gymnasium as gym

env = gym.make("Ant-v4", render_mode="human")  # replace with HalfCheetah-v4 or Walker2d-v4
obs = env.reset()

for _ in range(1000):
    env.render()
    action = env.action_space.sample()  # random action
    obs, reward, terminated, truncated, info = env.step(action)
    
    done = terminated or truncated
    if done:
        obs = env.reset()

env.close()

/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


In [10]:
print(env.observation_space)

Box(-inf, inf, (27,), float64)


In [11]:
print(env.action_space)

Box(-1.0, 1.0, (8,), float32)


In [18]:
import mujoco
import mujoco.viewer
import numpy as np

# Load XML model
model = mujoco.MjModel.from_xml_path("ant.xml")  # download Ant XML from Gym's mujoco_models
data = mujoco.MjData(model)

viewer = mujoco.viewer.launch_passive(model,data)

for _ in range(1000):
    mujoco.mj_step(model, data)  # advance physics

    # Get CoM of entire robot
    com = data.subtree_com[0]  # index 0 = root
    print("CoM:", com)

    # Optional: check velocity of CoM
    com_vel = data.subtree_linvel[0]
    print("CoM velocity:", com_vel)

    viewer.render()


CoM: [ 3.27040642e-21 -3.29517377e-21  7.49509990e-01]
CoM velocity: [0. 0. 0.]


AttributeError: 'Handle' object has no attribute 'render'